# InsTAP

<img src="http://store-images.s-microsoft.com/image/apps.31997.13510798887167234.6cd52261-a276-49cf-9b6b-9ef8491fb799.30e70ce4-33c5-43d6-9af1-491fe4679377" alt="Drawing" style="width: 200px;" align="center" />

Il progetto **InsTAP** ha come obiettivo quello di analizzare diversi profili Instagram e valutare la positività o negatività dei commenti presenti nei vari post.

Coded by Gianluca Di Franco e Rosario Amantia

[Github Project](https://github.com/rosarioamantia/insTAP)

*Immagine del flusso del proggetti*
   

# Data Ingestion

La sorgente è ovviamente il Social Network Instagram. Per recuperare i dati di cui avevamo bisogno, abbiamo fatto uso di un pacchetto chiamato **Instaloader**.

![download.jpg](images/instaloader_ko.jpg)

## Producer

Al fine di generare ed inviare i dati da Instagram abbiamo programmato un producer basato sul pacchetto instaloader citato in precedenza.

Il producer si occuperà di recuperare ed inviare le seguenti informazioni contenute all'interno dei post degli utenti in esame: 
- utente che crea il post
- Immagine 
- descrizione
- data di creazione
- coordinate
- commenti 

~~pip install instaloader~~

In [1]:
pip install --no-cache-dir git+https://github.com/IL2C/instaloader.git@login-fix

  Cloning https://github.com/IL2C/instaloader.git (to revision login-fix) to c:\users\gianluca\appdata\local\temp\pip-req-build-8sijk5mx
  Resolved https://github.com/IL2C/instaloader.git to commit 8f06668502ca4b90f1e9dff3e954e2146087115e
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/IL2C/instaloader.git 'C:\Users\Gianluca\AppData\Local\Temp\pip-req-build-8sijk5mx'
  Running command git checkout -b login-fix --track origin/login-fix
  Branch 'login-fix' set up to track remote branch 'login-fix' from 'origin'.
  Switched to a new branch 'login-fix'
You should consider upgrading via the 'd:\UNI\TAP\insTAP\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [8]:
import instaloader

insta = instaloader.Instaloader()
insta.login("user", "pass")
posts = instaloader.Profile.from_username(insta.context, "giorgiameloni").get_posts()

for i, post in enumerate(posts):
    if i == 3:
        break
    comments = post.get_comments()
    for index, comment in enumerate(comments):
        if index == 3:
            break

        data = {
            'message_id': index,
            'post_id': f'{post.owner_username}_{i}',
            'user': post.owner_username,
            'comment': comment.text,
            'caption': post.caption,
            'image': post.url,
            'timestamp': str(post.date_local),
            'likes': post.likes,
            'lat': post.location.lat if post.location else None,
            'lng': post.location.lng if post.location else None
        }
        print(str(data))

KeyError: "name='csrftoken', domain=None, path=None"

# Logstash

![download.jpg](images/logstash.jpg)

Logstash è uno strumento in grado di raccogliere, modificare e inoltrare dati provenienti da sorgenti diverse verso una o più destinazioni.
Il file di configurazione di logstash è formato da tre sezioni:
- **input**: ricevere/recuperare le informazioni;
- **filter**: rimuovere/aggiungere/modificare campi;
- **output**: inviare i dati alla destinazione. 

### il nostro .conf
```
input {
 http {
    port => "9700"
    host => "0.0.0.0"
  }
}

filter {
}

output {
  kafka {
    codec => json
    topic_id => "instap"
    bootstrap_servers => "http://broker:9092"
  }
}

```

Nel nostro caso, Logstash recupera i dati inviati dalla sorgente alla porta 9700 e li invia al broker di messaggi Kafka.

![logstash-explained.jpg](images/logstash-explained.jpg)

# Apache Kafka

![kafka.png](images/kafka.png)

Apache Kafka è una piattaforma distribuita basata sui messaggi e sul modello publish-subscribe con lo scopo di gestire streaming di dati.

Kafka è ideale per:
- Scambi di dati affidabili tra componenti diversi
- Gestione dei carichi di lavoro
- Streaming in tempo reale per l'elaborazione dei dati
- Supporto per la riproduzione di dati/messaggi

Concetti chiave di Kafka:
- **Producer**: colui che invia i dati. Il producer definisce anche su quale topic dovrà arrivare il messaggio.
- **Topic**: argomento del messaggio. 
    - i producers scelgono su quale topic devono inviare i messaggi;
    - i consumers scelgono su quale topic leggere i messaggi. 
- **Partizione**: struttura dati all'interno del topic in cui vengono scritti i messaggi. Ogni topic ha 1 o più partizioni.
- **Consumer**: colui che legge i messaggi. Ogni consumer appartiene ad un gruppo.

![consumer-kafka.png](images/consumer-group-kafka.png)

All'interno del nostro progetto, Kafka è così suddiviso:
- **Zookeper**: servizio centralizzatto per il mantenimento di configurazioni e denominazioni; 
- **Broker**: Cuore di Kafka, gestisce il flusso di dati proveniente da ElasticSearch;
- **Init-Kafka**: Script per l'inizializzazione di topic, partition..
- **Kafka-Ui**: interfaccia Web relativa a Kafka.

# Spark

<h3><img src="images/spark.png" width="430px" height="226px"> &nbsp;</h3>
<p>Apache Spark è un framework open source di elaborazione parallela che supporta l'elaborazione in memoria per migliorare le prestazioni delle applicazioni che analizzano Big Data.</p>

***migliorare descrizione***

<p>
Per l'allenamento del modello è stato utilizzato PySpark ed in particolare la libreria Spark MLlib utilizzando la funzione "LogisticRegression" all'interno di una pipeline di machine learning.
</p>

```
#tokenize the Instagram comments text    
stage_1 = RegexTokenizer(inputCol= 'comment' , outputCol= 'tokens', pattern= '\\W')

#remove the stop words
stage_2 = StopWordsRemover(inputCol= 'tokens', outputCol= 'filtered_words')

#create a word vector of size 50
stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector', vectorSize= 50)

#Logistic Regression Model
model = LogisticRegression(featuresCol= 'vector', labelCol= 'positive')

# setup the pipeline
pipeline = Pipeline(stages= [stage_1, stage_2, stage_3, model])

dataset = spark.read.csv('../tap/spark/dataset/social_training_set.csv',
                         schema=training_set_schema,
                         header=True,
                         sep=',')

#trained model
pipelineFit = pipeline.fit(dataset)

```

<br>
quando avviene la chiamata pipeline.fit(), viene adattato il modello al training set di input e gli stage vengono eseguiti in ordine per ogni record del dataset

<img src="images/before_training.jpg" alighn="center">

<img src="images/after_training_1.jpg" align="center">


Una volta ottenuto il modello allenato, il passo successivo è quello di leggere i dati inseriti in tempo reale da Kafka nel topic "instap_topic".
<br>
<br>
Per leggere il flusso di dati viene usata la libreria Spark Structured Streaming, una libreria per processare i dati in streaming all'interno della libreria Spark SQL
<br>

```

#Create DataFrame representing the stream of input lines from Kafka
df = spark.readStream.format('kafka')
    .option('kafka.bootstrap.servers', kafkaServer) \
        .option('subscribe', "instap_topic"). \
            option("startingOffsets","earliest").load()
```
<br>
<br>
<img src="images/kafka_spark.png" align="center">

<p>
   A questo punto, tramite un Output Structured Streaming, si elaborano i dati in stream applicando il modello sui dati di ogni micro-batch ed effettuando una predizione sul commento che verrà aggiunta in una colonna "prediction" nel DataFrame<br>
</p>

```
df.writeStream \
    .foreachBatch(process_batch) \
    .start() \
    .awaitTermination()
```
<br>

<img src="images/spark_ml.png" align="center">

Una volta processati e analizzati, i dati vengono inviati da PySpark verso Elasticsearch all'interno dell'index "instap_index" tramite il client Python

```
es = Elasticsearch(hosts=elastic_host, verify_certs = False)
resp = es.index(index = "instap_index", id = id, document = alayzed_data)

```
<br>
<img src="images/spark_es.png" align="center">


<img src="images/elasticsearch.png" align="center" width="500px" height="500px">
<p>
    Elasticsearch è un motore per la ricerca e l'analisi di dati: è in grado di gestire tutte le tipologie di dato (testuale, numerico, geospaziale, strutturato e non strutturato) ed è conosciuto per la sua natura distribuita, velocità e scalabilità.
<br>
<br>
Elasticsearch è stato utilizzato per indicizzare e memorizzare i dati precedentemente elaborati e inviarli a Kibana per visualizzarli.
</p>
<br>
<b>Nota: </b>una volta memorizzati, i dati  possono essere recuperati in modo veloce ed efficiente attraverso semplici chiamate REST o le apposite API

<p><b>Le conferme che rendono felici<b></p>
<img src="images/es_log.png">

<img src="images/kibana.png" width="430px" height="226px">

Kibana è una UI utilizzata per visualizzare ed analizzare i dati di Elasticsearch con l'ausilio di grafici. <br>
Di seguito vengono riportati alcuni grafici mostrati nella dashboard. <br>

![Homer-Kibana.jpg](images/Homer-kibana.jpg)

![kibana-dashboard.jpg](images/all-dashboard.jpg)

![kibana-post-prediction.jpg](images/kibana-post-prediction.jpg)

![kibana-donut.jpg](images/kibana-donut.jpg)